In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings                              # To ignore any warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.preprocessing import LabelEncoder     # import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [65]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from sklearn.ensemble import RandomForestClassifier
import xgboost
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.combine import SMOTETomek

In [4]:
train = pd.read_csv("train_LZdllcl.csv")
test = pd.read_csv("test_2umaH9m.csv")
submission = pd.read_csv("sample_submission_M0L0uXE.csv")

In [5]:
train_original = train.copy() 
test_original = test.copy()

In [6]:
display(train.head())
display(test.head())
display(submission.head())

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61


,employee_id,is_promoted
0,8724,0
1,74430,0
2,72255,0
3,38562,0
4,64486,0


In [7]:
display(train.shape)
display(test.shape)

(54808, 14)

(23490, 13)

In [8]:
display(train.info())
display(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54808 entries, 0 to 54807
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   employee_id           54808 non-null  int64  
 1   department            54808 non-null  object 
 2   region                54808 non-null  object 
 3   education             52399 non-null  object 
 4   gender                54808 non-null  object 
 5   recruitment_channel   54808 non-null  object 
 6   no_of_trainings       54808 non-null  int64  
 7   age                   54808 non-null  int64  
 8   previous_year_rating  50684 non-null  float64
 9   length_of_service     54808 non-null  int64  
 10  KPIs_met >80%         54808 non-null  int64  
 11  awards_won?           54808 non-null  int64  
 12  avg_training_score    54808 non-null  int64  
 13  is_promoted           54808 non-null  int64  
dtypes: float64(1), int64(8), object(5)
memory usage: 5.9+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23490 entries, 0 to 23489
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   employee_id           23490 non-null  int64  
 1   department            23490 non-null  object 
 2   region                23490 non-null  object 
 3   education             22456 non-null  object 
 4   gender                23490 non-null  object 
 5   recruitment_channel   23490 non-null  object 
 6   no_of_trainings       23490 non-null  int64  
 7   age                   23490 non-null  int64  
 8   previous_year_rating  21678 non-null  float64
 9   length_of_service     23490 non-null  int64  
 10  KPIs_met >80%         23490 non-null  int64  
 11  awards_won?           23490 non-null  int64  
 12  avg_training_score    23490 non-null  int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 2.3+ MB


None

In [9]:
train.drop(['employee_id'], axis=1, inplace=True)
test.drop(['employee_id'], axis=1, inplace=True)

### Missing Values

In [10]:
display(train.isnull().sum())
display(test.isnull().sum())

department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

department                 0
region                     0
education               1034
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    1812
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
dtype: int64

In [11]:
train['education'].value_counts()

Bachelor's          36669
Master's & above    14925
Below Secondary       805
Name: education, dtype: int64

In [12]:
train['previous_year_rating'].value_counts()

3.0    18618
5.0    11741
4.0     9877
1.0     6223
2.0     4225
Name: previous_year_rating, dtype: int64

In [13]:
train['education'] = train['education'].fillna("secondary")
test['education'] = test['education'].fillna("secondary")

In [14]:
train['previous_year_rating'] = train['previous_year_rating'].fillna(3.0)
test['previous_year_rating'] = test['previous_year_rating'].fillna(3.0)

<h2 style="color:blue" align="left"> EDA (Exploratory Data Analysis) </h2>

### 1. Department

In [15]:
display(train['department'].value_counts())
display(test['department'].value_counts())

Sales & Marketing    16840
Operations           11348
Technology            7138
Procurement           7138
Analytics             5352
Finance               2536
HR                    2418
Legal                 1039
R&D                    999
Name: department, dtype: int64

Sales & Marketing    7315
Operations           4764
Procurement          3020
Technology           3011
Analytics            2319
Finance              1091
HR                   1085
Legal                 445
R&D                   440
Name: department, dtype: int64

In [16]:
display(train['department'].nunique())
display(test['department'].nunique())

9

9

- Here we have **9 unique labels**. And **there is no ordering** so it is a **nominal category**.

In [17]:
train_department = pd.get_dummies(train['department'], drop_first=True) # apply pd.get_dummies
train_department.head(10)

,Finance,HR,Legal,Operations,Procurement,R&D,Sales & Marketing,Technology
0,0,0,0,0,0,0,1,0
1,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0
6,0,0,0,1,0,0,0,0
7,0,0,0,1,0,0,0,0
8,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,1,0


In [18]:
test_department = pd.get_dummies(test['department'], drop_first=True) # apply pd.get_dummies
test_department.head(10)

,Finance,HR,Legal,Operations,Procurement,R&D,Sales & Marketing,Technology
0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0
3,0,0,0,0,1,0,0,0
4,1,0,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0
6,1,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,1


In [19]:
train.drop(['department'], axis=1, inplace=True)
test.drop(['department'], axis=1, inplace=True)

### 2. Region

In [20]:
display(train['region'].value_counts())
display(test['region'].value_counts())

region_2     12343
region_22     6428
region_7      4843
region_15     2808
region_13     2648
region_26     2260
region_31     1935
region_4      1703
region_27     1659
region_16     1465
region_28     1318
region_11     1315
region_23     1175
region_29      994
region_32      945
region_19      874
region_20      850
region_14      827
region_25      819
region_17      796
region_5       766
region_6       690
region_30      657
region_8       655
region_10      648
region_1       610
region_24      508
region_12      500
region_9       420
region_21      411
region_3       346
region_34      292
region_33      269
region_18       31
Name: region, dtype: int64

region_2     5299
region_22    2739
region_7     1982
region_13    1167
region_15    1130
region_26    1011
region_31     844
region_4      775
region_27     710
region_28     595
region_16     590
region_11     571
region_23     516
region_32     433
region_29     414
region_19     410
region_17     361
region_14     350
region_5      342
region_25     337
region_20     326
region_6      298
region_30     273
region_10     269
region_8      269
region_1      238
region_24     219
region_12     215
region_9      180
region_21     179
region_34     155
region_3      147
region_33     126
region_18      20
Name: region, dtype: int64

In [21]:
display(train['region'].nunique())
display(test['region'].nunique())

34

34

In [22]:
le = LabelEncoder()
train['region_en'] = le.fit_transform(train['region'])
train['region_en']

0        31
1        14
2        10
3        15
4        18
         ..
54803     5
54804    19
54805     0
54806    33
54807    14
Name: region_en, Length: 54808, dtype: int32

In [23]:
region_en_train = pd.DataFrame(train['region_en'])
region_en_train

,region_en
0,31
1,14
2,10
3,15
4,18
...,...
54803,5
54804,19
54805,0
54806,33


In [24]:
test['region_en'] = le.fit_transform(test['region'])
test['region_en']

0        18
1        28
2         4
3        11
4        21
         ..
23485    11
23486    17
23487     7
23488    24
23489     8
Name: region_en, Length: 23490, dtype: int32

In [25]:
region_en_test = pd.DataFrame(test['region_en'])
region_en_test

,region_en
0,18
1,28
2,4
3,11
4,21
...,...
23485,11
23486,17
23487,7
23488,24


In [26]:
train.drop('region', axis=1, inplace=True)
test.drop('region', axis=1, inplace=True)

### Education

In [27]:
display(train['education'].value_counts())
display(test['education'].value_counts())

Bachelor's          36669
Master's & above    14925
secondary            2409
Below Secondary       805
Name: education, dtype: int64

Bachelor's          15578
Master's & above     6504
secondary            1034
Below Secondary       374
Name: education, dtype: int64

In [30]:
train['education_en'] = le.fit_transform(train['education'])
train['education_en']

0        2
1        0
2        0
3        0
4        0
        ..
54803    0
54804    2
54805    0
54806    3
54807    0
Name: education_en, Length: 54808, dtype: int32

In [31]:
education_en_train = pd.DataFrame(train['education_en'])
education_en_train

,education_en
0,2
1,0
2,0
3,0
4,0
...,...
54803,0
54804,2
54805,0
54806,3


In [32]:
test['education_en'] = le.fit_transform(test['education'])
test['education_en']

0        0
1        0
2        0
3        0
4        0
        ..
23485    1
23486    0
23487    0
23488    0
23489    2
Name: education_en, Length: 23490, dtype: int32

In [33]:
education_en_test = pd.DataFrame(test['education_en'])
education_en_test

,education_en
0,0
1,0
2,0
3,0
4,0
...,...
23485,1
23486,0
23487,0
23488,0


In [34]:
train.drop('education', axis=1, inplace=True)
test.drop('education', axis=1, inplace=True)

### Gender

In [35]:
display(train['gender'].value_counts())
display(test['gender'].value_counts())

m    38496
f    16312
Name: gender, dtype: int64

m    16596
f     6894
Name: gender, dtype: int64

In [36]:
train['gender'] = train['gender'].map({'m':1, 'f':0})
test['gender'] = test['gender'].map({'m':1, 'f':0})

In [37]:
display(train['gender'].value_counts())
display(test['gender'].value_counts())

1    38496
0    16312
Name: gender, dtype: int64

1    16596
0     6894
Name: gender, dtype: int64

In [39]:
display(train.head())
display(test.head())

,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,region_en,education_en
0,0,sourcing,1,35,5.0,8,1,0,49,0,31,2
1,1,other,1,30,5.0,4,0,0,60,0,14,0
2,1,sourcing,1,34,3.0,7,0,0,50,0,10,0
3,1,other,2,39,1.0,10,0,0,50,0,15,0
4,1,other,1,45,3.0,2,0,0,73,0,18,0


,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,region_en,education_en
0,1,sourcing,1,24,3.0,1,1,0,77,18,0
1,0,other,1,31,3.0,5,0,0,51,28,0
2,1,other,1,31,1.0,4,0,0,47,4,0
3,0,other,3,31,2.0,9,0,0,65,11,0
4,1,sourcing,1,30,4.0,7,0,0,61,21,0


### Recruitment_channel

In [38]:
display(train['recruitment_channel'].value_counts())
display(test['recruitment_channel'].value_counts())

other       30446
sourcing    23220
referred     1142
Name: recruitment_channel, dtype: int64

other       13078
sourcing     9961
referred      451
Name: recruitment_channel, dtype: int64

In [40]:
train['recruitment_channel'] = train['recruitment_channel'].map({'sourcing':0, 'referred':1, 'other':2})
test['recruitment_channel'] = test['recruitment_channel'].map({'sourcing':1, 'referred':1, 'other':2})

In [41]:
display(train['recruitment_channel'].value_counts())
display(test['recruitment_channel'].value_counts())

2    30446
0    23220
1     1142
Name: recruitment_channel, dtype: int64

2    13078
1    10412
Name: recruitment_channel, dtype: int64

In [42]:
display(train.head())
display(test.head())

,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,region_en,education_en
0,0,0,1,35,5.0,8,1,0,49,0,31,2
1,1,2,1,30,5.0,4,0,0,60,0,14,0
2,1,0,1,34,3.0,7,0,0,50,0,10,0
3,1,2,2,39,1.0,10,0,0,50,0,15,0
4,1,2,1,45,3.0,2,0,0,73,0,18,0


,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,region_en,education_en
0,1,1,1,24,3.0,1,1,0,77,18,0
1,0,2,1,31,3.0,5,0,0,51,28,0
2,1,2,1,31,1.0,4,0,0,47,4,0
3,0,2,3,31,2.0,9,0,0,65,11,0
4,1,1,1,30,4.0,7,0,0,61,21,0


### is_promoted

In [43]:
train['is_promoted'].value_counts()

0    50140
1     4668
Name: is_promoted, dtype: int64

In [44]:
train_final = pd.concat([train, train_department, region_en_train, education_en_train], axis=1)
test_final = pd.concat([test, test_department, region_en_test, education_en_test], axis=1)

In [45]:
display(train_final.head())
display(test_final.head())

,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,...,Finance,HR,Legal,Operations,Procurement,R&D,Sales & Marketing,Technology,region_en,education_en
0,0,0,1,35,5.0,8,1,0,49,0,...,0,0,0,0,0,0,1,0,31,2
1,1,2,1,30,5.0,4,0,0,60,0,...,0,0,0,1,0,0,0,0,14,0
2,1,0,1,34,3.0,7,0,0,50,0,...,0,0,0,0,0,0,1,0,10,0
3,1,2,2,39,1.0,10,0,0,50,0,...,0,0,0,0,0,0,1,0,15,0
4,1,2,1,45,3.0,2,0,0,73,0,...,0,0,0,0,0,0,0,1,18,0


,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,region_en,...,Finance,HR,Legal,Operations,Procurement,R&D,Sales & Marketing,Technology,region_en,education_en
0,1,1,1,24,3.0,1,1,0,77,18,...,0,0,0,0,0,0,0,1,18,0
1,0,2,1,31,3.0,5,0,0,51,28,...,0,1,0,0,0,0,0,0,28,0
2,1,2,1,31,1.0,4,0,0,47,4,...,0,0,0,0,0,0,1,0,4,0
3,0,2,3,31,2.0,9,0,0,65,11,...,0,0,0,0,1,0,0,0,11,0
4,1,1,1,30,4.0,7,0,0,61,21,...,1,0,0,0,0,0,0,0,21,0


In [46]:
display(train_final.shape)
display(test_final.shape)

(54808, 22)

(23490, 21)

<h2 style="color:blue" align="left"> Model building and Evaluation </h2>

In [47]:
# Independant variable
X = train_final.drop('is_promoted',axis=1)        # All rows & columns exclude Target features

# Dependant variable
y = train_final['is_promoted']                   # Only target feature

In [48]:
# split  data into training and testing sets of 80:20 ratio
# 20% of test size selected
# random_state is random seed
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=4)

In [49]:
# shape of X & Y test / train
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(43846, 21) (10962, 21) (43846,) (10962,)


### SMOTE

In [80]:
from collections import Counter

In [81]:
os = SMOTETomek(0.75)
X_train_ns,y_train_ns = os.fit_sample(X_train,y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_ns)))

The number of classes before fit Counter({0: 40081, 1: 3765})
The number of classes after fit Counter({0: 39959, 1: 29938})


### 1. Logistic Regression

In [52]:
LogReg = LogisticRegression(solver='liblinear', multi_class='ovr')
LogReg.fit(X_train, y_train)
LogReg.score(X_test, y_test)

0.9328589673417259

In [53]:
y_pred_LogReg = LogReg.predict(X_test)

In [54]:
print("Train Score {:.2f} & Test Score {:.2f}".format(LogReg.score(X_train, y_train), LogReg.score(X_test, y_test)))

Train Score 0.93 & Test Score 0.93


In [55]:
accuracy_score(y_test, y_pred_LogReg)

0.9328589673417259

### 2. Random Forest Classifier

In [58]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

RandomForestClassifier()

In [59]:
pred_rf = rf.predict(X_test)

In [60]:
print("Train Score {:.2f} & Test Score {:.2f}".format(rf.score(X_train, y_train), rf.score(X_test, y_test)))

Train Score 1.00 & Test Score 0.94


In [61]:
accuracy_score(y_test, pred_rf )

0.9386060937785076

In [85]:
classifier = RandomForestClassifier()
classifier.fit(X_train_ns,y_train_ns)

RandomForestClassifier()

In [86]:
pred_rf_smote = classifier.predict(X_test)

In [87]:
print("Train Score {:.2f} & Test Score {:.2f}".format(classifier.score(X_train_ns, y_train_ns), classifier.score(X_test, y_test)))

Train Score 1.00 & Test Score 0.93


### 3. XGBoost Classifier

In [94]:
classifier1 = XGBClassifier(colsample_bytree=0.5, gamma=0.4, learning_rate=0.1,
                           max_delta_step=0, max_depth=10, min_child_weight=9,
                           booster='gbtree', n_estimators=300, objective='multi:softprob', random_state=100)

classifier1.fit(X_train,y_train)

ValueError: feature_names must be unique

In [93]:
reg_xgb = XGBClassifier(objective='binary:logistic')
reg_xgb.fit(X_train, y_train)

ValueError: feature_names must be unique

In [ ]:
# predicting X_test
y_pred_xgb = reg_xgb.predict(X_test)

In [ ]:
print("Train Score {:.2f} & Test Score {:.2f}".format(reg_xgb.score(X_train,y_train),reg_xgb.score(X_test,y_test)))

In [ ]:
accuracy_score(y_test, y_pred_xgb)

### 4. LGBM

In [101]:
from lightgbm import LGBMClassifier

lgbm_model = LGBMClassifier(objective='binary', num_leaves=31, learning_rate=0.01, n_estimators=10000,
                            max_bin = 100, bagging_fraction = 0.8, bagging_freq = 5, feature_fraction = 0.8,
                            feature_fraction_seed=9, bagging_seed=9, min_data_in_leaf =100, min_sum_hessian_in_leaf = 11)
lgbm_model = lgbm_model.fit(X_train_ns, y_train_ns)      

In [102]:
y_pred_LGBM = lgbm_model.predict(X_test)

In [103]:
print("Train Score {:.2f} & Test Score {:.2f}".format(lgbm_model.score(X_train,y_train),lgbm_model.score(X_test,y_test)))

Train Score 0.96 & Test Score 0.94


### 5. Gradient Boosting Classifier

In [106]:
from sklearn.ensemble import GradientBoostingClassifier
GBC = GradientBoostingClassifier(n_estimators=300)
GBC.fit(X_train, y_train)

GradientBoostingClassifier(n_estimators=300)

In [107]:
y_pred_GBC = GBC.predict(X_test)

In [108]:
print("Train Score {:.2f} & Test Score {:.2f}".format(GBC.score(X_train,y_train), GBC.score(X_test,y_test)))

Train Score 0.94 & Test Score 0.94


### Submission

In [109]:
# predicting X_test
y_pred_test = GBC.predict(test_final)

In [110]:
submission = pd.DataFrame({'employee_id': test_original['employee_id'], 'is_promoted': y_pred_test})
submission.to_csv('HR Analytics.csv', index=False)